In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas_profiling import ProfileReport
import pickle as pk
import optuna

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_20/1964004205.py:7: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
ex = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")
ex.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [3]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [4]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
# file = ProfileReport(train, title="Spaceship Titanic")

In [6]:
# file.to_notebook_iframe()

In [7]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [8]:
train.shape

(8693, 14)

In [9]:
for col in train:
    print(col, train[col].isna().sum())

PassengerId 0
HomePlanet 201
CryoSleep 217
Cabin 199
Destination 182
Age 179
VIP 203
RoomService 181
FoodCourt 183
ShoppingMall 208
Spa 183
VRDeck 188
Name 200
Transported 0


In [10]:
temp = train

temp.dropna(inplace=True)

temp.shape

(6606, 14)

In [11]:
def preprocessing(df):
    
    print(df.columns)
    
    ##########################################
    
    group_num = []
    group_pos = []

    for entry in df['PassengerId']:
        entry = entry.split("_")
        
        group_num.append(entry[0])
        group_pos.append(entry[1])

    df.drop(labels=['PassengerId'], inplace=True, axis = 1)

    df['Group_num'] = group_num
    df['group_pos'] = group_pos
    
    ##########################################
    
    deck = []
    num = []
    side = []
    
    for entry in df['Cabin']:
        entry = entry.split("/")

        deck.append(entry[0])
        num.append(entry[1])
        side.append(entry[2])
    
    df.drop(labels=['Cabin'], inplace=True, axis = 1)

    df['deck'] = deck
    df['num'] = num
    df['side'] = side
    
    ##########################################
    
    total_spent = []
    
    for i in range(df.shape[0]):
        total_spent.append(df.iloc[i]['RoomService'] + df.iloc[i]['FoodCourt'] + df.iloc[i]['ShoppingMall'] + df.iloc[i]['Spa'] + df.iloc[i]['VRDeck'])
    
    df['total_spent'] = total_spent
    
    ##########################################
    
    # Factorize all Non-Numerical Entries
    
    for col in df:
        if df[col].dtype == 'object':
            df[col] = df[col].factorize()[0]
    
    ##########################################
    
    df['Transported'] = df['Transported'].factorize()[0]
    
    ##########################################
    
    print(df.head())
    
    return df

In [12]:
temp = preprocessing(temp)

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')
   HomePlanet  CryoSleep  Destination   Age  VIP  RoomService  FoodCourt  \
0           0          0            0  39.0    0          0.0        0.0   
1           1          0            0  24.0    0        109.0        9.0   
2           0          0            0  58.0    1         43.0     3576.0   
3           0          0            0  33.0    0          0.0     1283.0   
4           1          0            0  16.0    0        303.0       70.0   

   ShoppingMall     Spa  VRDeck  Name  Transported  Group_num  group_pos  \
0           0.0     0.0     0.0     0            0          0          0   
1          25.0   549.0    44.0     1            1          1          0   
2           0.0  6715.0    49.0     2            0          2          0   
3         371.0  3329.0   193.0

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(temp.iloc[:,temp.columns != 'Transported'], temp['Transported'], test_size=0.3)

In [14]:
# Establish Baseline Accuracy of 79%

from sklearn.neighbors import KNeighborsClassifier

for i in range(1, 200):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)

    preds = knn.predict(X_valid)
    
    num_correct = 0
    
    for i, pred in enumerate(preds):
        if pred == y_valid.iloc[i]:
            num_correct += 1 
    
    print(num_correct / len(preds))
    

0.7103935418768921
0.7149344096871847
0.756811301715439
0.7502522704339052
0.7704339051463168
0.7674066599394551
0.7729566094853683
0.7704339051463168
0.7785065590312815
0.7764883955600403
0.7820383451059536
0.7805247225025227
0.7800201816347124
0.7764883955600403
0.7840565085771948
0.7749747729566094
0.7780020181634713
0.7830474268415741
0.7865792129162462
0.7820383451059536
0.7850655903128153
0.7850655903128153
0.786074672048436
0.7820383451059536
0.7850655903128153
0.7815338042381433
0.7840565085771948
0.784561049445005
0.7870837537840565
0.786074672048436
0.7865792129162462
0.7880928355196771
0.7885973763874874
0.7865792129162462
0.7880928355196771
0.7840565085771948
0.7870837537840565
0.7896064581231079
0.7885973763874874
0.7875882946518667
0.7891019172552977
0.7911200807265388
0.7911200807265388
0.7921291624621595
0.79313824419778
0.7880928355196771
0.7906155398587286
0.7906155398587286
0.7926337033299697
0.7906155398587286
0.7901109989909183
0.7891019172552977
0.7875882946518667

In [15]:
# Base Accuracy: 75.17% | HyperTuned Model's Accuracy: ~78.86%

from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)

preds = dtree.predict(X_valid)
    
num_correct = 0

for i, pred in enumerate(preds):
    if pred == y_valid.iloc[i]:
        num_correct += 1 

print(num_correct / len(preds))

0.7522704339051464


In [16]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [17]:
criterions = ['gini', 'entropy', 'log_loss']
splitters = ['best', 'random']
max_depths = [i for i in range(1, 20)]
min_samples_splits = [i for i in range(2, 10)]
min_samples_leafs = [i for i in range(1, 10)]
max_features = ['auto', 'sqrt', 'log2']

compiled_models = []

i = 0

for criterion in criterions:
    for splitter in splitters:
        for max_depth in max_depths:
            for min_samples_split in min_samples_splits:
                for min_samples_leaf in min_samples_leafs:
                    for max_feature in max_features:
                        
                        try:
                            
                            model = DecisionTreeClassifier(criterion=criterion, splitter=splitter, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_features=max_feature)
                            model.fit(X_train, y_train)

                            compiled_models.append([float(model.score(X_valid,y_valid)), model])

                        except:
                            pass
                        
                        i += 1
                        
                        if i % 1000 == 0:
                            print(i)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


In [18]:
n = 20

compiled_models.sort(key=lambda row: (row[0]))
compiled_models = compiled_models[::-1]
compiled_models = compiled_models[0:n]

for i, model in enumerate(compiled_models):
    print(model[0], model[1])
    filename = f"DecisionTree_{i}.pickle"
    pk.dump(model[1], open(filename, 'wb'))

0.7946518668012109 DecisionTreeClassifier(max_depth=9, max_features='log2', min_samples_leaf=7,
                       min_samples_split=5)
0.7936427850655903 DecisionTreeClassifier(criterion='log_loss', max_depth=19, max_features='log2',
                       min_samples_leaf=9, min_samples_split=6,
                       splitter='random')
0.7936427850655903 DecisionTreeClassifier(criterion='entropy', max_depth=10, max_features='auto',
                       min_samples_leaf=4, min_samples_split=5)
0.79313824419778 DecisionTreeClassifier(max_depth=8, max_features='auto', min_samples_leaf=8,
                       min_samples_split=7)
0.7926337033299697 DecisionTreeClassifier(criterion='log_loss', max_depth=9, max_features='auto',
                       min_samples_leaf=3, min_samples_split=9)
0.7926337033299697 DecisionTreeClassifier(criterion='entropy', max_depth=16, max_features='sqrt',
                       min_samples_leaf=4, min_samples_split=7)
0.7921291624621595 DecisionTree

In [19]:
# Base Accuracy: 80.17% | HyperTuned Model's Accuracy: 

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

model.score(X_valid, y_valid)

0.8037336024217961

In [20]:
def objective(trial):
    
    # Split the Data Each time
    X_train, X_valid, y_train, y_valid = train_test_split(temp.iloc[:,temp.columns != 'Transported'], temp['Transported'], test_size=0.3)
    
    # Set the Parameters
    n_estimator = trial.suggest_int('n_estimators', 2, 400)
    max_depth = int(trial.suggest_int('max_depth', 1, 32))
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])
    min_samples_split = int(trial.suggest_int('min_samples_split', 2, 10))
    min_samples_leaf = int(trial.suggest_int('min_samples_leaf', 1, 10))
    max_feature = trial.suggest_categorical('max_feature', ['auto', 'sqrt', 'log2'])
    
    try:
        model = RandomForestClassifier(n_estimators=n_estimator, criterion=criterion, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_features=max_feature)
        model.fit(X_train, y_train)

        return model.score(X_valid, y_valid)
    
    except:
        
        return 0

In [21]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=400)

[I 2023-05-22 12:06:02,057] A new study created in memory with name: no-name-ff20dc74-485c-41df-8593-a5ef9e2ea474
[I 2023-05-22 12:06:02,314] Trial 0 finished with value: 0.798183652875883 and parameters: {'n_estimators': 35, 'max_depth': 8, 'criterion': 'gini', 'min_samples_split': 7, 'min_samples_leaf': 7, 'max_feature': 'sqrt'}. Best is trial 0 with value: 0.798183652875883.
[I 2023-05-22 12:06:04,647] Trial 1 finished with value: 0.8082744702320888 and parameters: {'n_estimators': 302, 'max_depth': 15, 'criterion': 'gini', 'min_samples_split': 8, 'min_samples_leaf': 9, 'max_feature': 'sqrt'}. Best is trial 1 with value: 0.8082744702320888.
[I 2023-05-22 12:06:07,487] Trial 2 finished with value: 0.8092835519677094 and parameters: {'n_estimators': 376, 'max_depth': 19, 'criterion': 'gini', 'min_samples_split': 3, 'min_samples_leaf': 9, 'max_feature': 'auto'}. Best is trial 2 with value: 0.8092835519677094.
[I 2023-05-22 12:06:09,017] Trial 3 finished with value: 0.8022199798183652 a

In [22]:
study.best_params

{'n_estimators': 276,
 'max_depth': 31,
 'criterion': 'entropy',
 'min_samples_split': 3,
 'min_samples_leaf': 3,
 'max_feature': 'auto'}

In [23]:
# Base Accuracy: 79.41% | Hyper-Tuned: ~ 82%

import xgboost as xgb

model = xgb.XGBClassifier()
model.fit(X_train, y_train)

model.score(X_valid, y_valid)

0.8092835519677094

In [24]:
def objective(trial):
        
    # Split the Data Each time
    X_train, X_valid, y_train, y_valid = train_test_split(temp.iloc[:,temp.columns != 'Transported'], temp['Transported'], test_size=0.3)
    
    # Set the Parameters
    colsample_bytree = trial.suggest_uniform("colsample_bytree", 0.3, 0.7)
    gamma = trial.suggest_uniform("gamma", 0, 0.5)
    learning_rate = trial.suggest_uniform("learning_rate", 0.03, 0.3)
    max_depth = int(trial.suggest_int("max_depth", 2, 6))
    n_estimators = int(trial.suggest_int("n_estimators", 100, 150))
    subsample = trial.suggest_float("subsample", 0.4, 0.6)
    
    try:
        model = xgb.XGBClassifier(colsample_bytree=colsample_bytree, gamma=gamma, learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators, subsample=subsample)
        model.fit(X_train, y_train)

        preds = model.predict(X_valid)
        
        num_correct = 0
        
        for i, pred in enumerate(preds):
            if pred == y_valid.iloc[i]:
                num_correct += 1
        
        return (num_correct / len(preds)) * 100
    
    except:
        
        return 0

In [25]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=400)

[I 2023-05-22 12:20:33,004] A new study created in memory with name: no-name-b4321696-dd0f-4225-9b89-6901f3500f80
[I 2023-05-22 12:20:33,211] Trial 0 finished with value: 80.62563067608475 and parameters: {'colsample_bytree': 0.5114409003284355, 'gamma': 0.12663103095230022, 'learning_rate': 0.2167029018023419, 'max_depth': 3, 'n_estimators': 113, 'subsample': 0.5482769748415497}. Best is trial 0 with value: 80.62563067608475.
[I 2023-05-22 12:20:33,513] Trial 1 finished with value: 80.32290615539858 and parameters: {'colsample_bytree': 0.4753418013403524, 'gamma': 0.013672622243089239, 'learning_rate': 0.15349703740156886, 'max_depth': 4, 'n_estimators': 146, 'subsample': 0.45859771344351674}. Best is trial 0 with value: 80.62563067608475.
[I 2023-05-22 12:20:33,714] Trial 2 finished with value: 79.56609485368314 and parameters: {'colsample_bytree': 0.47889351217274934, 'gamma': 0.4577881262462736, 'learning_rate': 0.04559193143474078, 'max_depth': 2, 'n_estimators': 146, 'subsample':

In [26]:
study.best_params

{'colsample_bytree': 0.3003332955924245,
 'gamma': 0.18221325512759973,
 'learning_rate': 0.06215312850908358,
 'max_depth': 4,
 'n_estimators': 145,
 'subsample': 0.5757972652045857}

In [27]:
# Base Accuracy: 80.37% | Hyper-Tuned Model: 82.48%

from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()

model.fit(X_train, y_train)

model.score(X_valid, y_valid)

0.8092835519677094

In [28]:
def objective(trial):
        
    # Split the Data Each time
    X_train, X_valid, y_train, y_valid = train_test_split(temp.iloc[:,temp.columns != 'Transported'], temp['Transported'], test_size=0.3)
    
    # Set the Parameters
    learning_rate = trial.suggest_uniform("learning_rate", 0.03, 0.3)
    max_depth = int(trial.suggest_int("max_depth", 2, 6))
    n_estimators = int(trial.suggest_int("n_estimators", 100, 150))
    
    try:
        model = GradientBoostingClassifier(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators)
        model.fit(X_train, y_train)

        return model.score(X_valid, y_valid)

    except:
        
        return 0

In [29]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=400)

[I 2023-05-22 12:22:50,727] A new study created in memory with name: no-name-9ae42598-a97e-4b43-b3b7-413f9a92dc22
[I 2023-05-22 12:22:51,838] Trial 0 finished with value: 0.7921291624621595 and parameters: {'learning_rate': 0.046213643801956965, 'max_depth': 2, 'n_estimators': 140}. Best is trial 0 with value: 0.7921291624621595.
[I 2023-05-22 12:22:53,547] Trial 1 finished with value: 0.7936427850655903 and parameters: {'learning_rate': 0.291173854012412, 'max_depth': 4, 'n_estimators': 117}. Best is trial 1 with value: 0.7936427850655903.
[I 2023-05-22 12:22:55,054] Trial 2 finished with value: 0.7870837537840565 and parameters: {'learning_rate': 0.12543251919801524, 'max_depth': 3, 'n_estimators': 132}. Best is trial 1 with value: 0.7936427850655903.
[I 2023-05-22 12:22:56,214] Trial 3 finished with value: 0.8027245206861756 and parameters: {'learning_rate': 0.17025027095493053, 'max_depth': 2, 'n_estimators': 143}. Best is trial 3 with value: 0.8027245206861756.
[I 2023-05-22 12:22

In [30]:
study.best_params

{'learning_rate': 0.0518127833455414, 'max_depth': 6, 'n_estimators': 107}

In [31]:
from sklearn import preprocessing

x = train.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
train_norm = pd.DataFrame(x_scaled)

In [32]:
# Split the Data Each time
X_train, X_valid, y_train, y_valid = train_test_split(train_norm.iloc[:,train_norm.columns != 11], train_norm[11], test_size=0.3)

In [33]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential, layers

In [34]:
model = Sequential()

model.add(layers.Dense(17, activation='relu', input_shape=(17,)))

model.add(layers.Dense(17, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy')

model.fit(X_train, y_train, batch_size = 32, epochs = 20, validation_data = (X_valid, y_valid))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 17)                306       
                                                                 
 dense_1 (Dense)             (None, 17)                306       
                                                                 
 dense_2 (Dense)             (None, 1)                 18        
                                                                 
Total params: 630
Trainable params: 630
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
145/145 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/20
145/145 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/20
145/145 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+0